# RNN generator inference

In [0]:
from pathlib import Path
dataset_path = Path.home() / 'data/rnn_generator'; dataset_path
tmp_path = dataset_path / 'tmp/'

In [0]:
ALLCHARS, MODEL = ['all_characters', 'model']

fn_witkacy = {ALLCHARS: 'all_characters.p', MODEL: 'model.h100.l1.e2000.cpu.dat'}
fn_pan_tadeusz = {ALLCHARS: 'all_characters.pan_tadeusz.p', MODEL: 'pan_tadeusz.h100.l1.e2000.cpu.torch'}

fn_dict = fn_pan_tadeusz

## Load all_characters list

In [5]:
import pickle

all_characters = pickle.load( open( tmp_path / fn_dict[ALLCHARS], 'rb' ) ); print(all_characters[:10])
n_characters = len(all_characters); print(n_characters)

['\n', ' ', '!', '"', '%', "'", '(', ')', ',', '-']
89


## Code

In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        input = self.encoder(input.view(1, -1))
        output, hidden = self.gru(input.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden

    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))

In [7]:
# Turn string into list of longs
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return Variable(tensor)

print(char_tensor('ala ma kota'))

Variable containing:
 41
 52
 41
  1
 53
 41
  1
 51
 55
 59
 41
[torch.LongTensor of size 11]



In [0]:
def evaluate(prime_str='A', predict_len=100, temperature=0.8):
    hidden = decoder.init_hidden()
    prime_input = char_tensor(prime_str)
    predicted = prime_str

    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = decoder(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = decoder(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char_tensor(predicted_char)

    return predicted

In [9]:
n_epochs = 300 # 3000
print_every = 100
plot_every = 1
hidden_size = 100
n_layers = 1
lr = 0.005

decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
print(decoder, flush=True)

RNN(
  (encoder): Embedding(89, 100)
  (gru): GRU(100, 100)
  (decoder): Linear(in_features=100, out_features=89)
)


## Load model

In [0]:
model_path_cpu = tmp_path / fn_dict[MODEL]
decoder = torch.load(model_path_cpu)

## Generate text

In [0]:
# priming strings
# pan_tadeusz: kon, bę, Tad, Tadeusz

In [25]:
print(evaluate('Tadeusz', 1000, temperature=0.8))

Tadeusz grzybę wipierzenanie tylcz które tylko za obył --
          czarał się szlicę mi przedziem ruchy w zawaczasz Bernarek zwodził zamku poronia,
Tele wielki nie wracie się zród krzykle w rozmusu;
Razem się barowowanie rozeszekłe do razy mu sładzi;
Myśliwsze przewy że się lasska.
Wiadła z obił wszyscy aż się już wieje czarku w zniessie się w prawy nawyklich,
Ustał wpod chybianéj siebie rozmiwszy prawie tabówka,
O jelent był oniskiemi młoniem, Gobnika,
Obiasił poświaczenie gniecz wypilne ledwienia
Lecz w ludziło i na szlecz opisowanie w ręcz rupierku za niéj porótkowali osporzasu,
Te wiejskim s przedżał w rękę między w nich -- Mowby przez więcéry weszcierzy,
To wostéj, ot onie lazem, lecia ruchomieni,
Więc Pan zręką, jakby zam powieki,
Róch jako nich ruz w ludzienie:
Ze w kłębow! Litwie się na niemy tylko szukiek i posława,
Goko się w się naprawie strzelecki bawiący i ręką rośmił się zawłaczą.
Ja sznom lasem puszcze krokie stary rozaje waję,
Obrawać co szlecha samom wszyscy! rzasły w

In [23]:
print(evaluate('Tad', 1000, temperature=0.2))

Tadem w niemyśli w koniec w ręku w to się do strzelcy w niemyśli się w ręką się w rękę przechodzić w rozmowy.
O nie zawszystkiem w koniec w ręku się w pod niemienie,
Ostał w nie się w krzyknął w przed rozmowy,
Odbierał się w konie w końce się w czysie się w podzierciał się w ręku w konie podobny za powierzy się w konie
Sito się w przed zawieli w ręku w przechodził w koniewszy się w lasu.

Tadeusz w koło w przechodzić w konie zawszystkienie się w konieckę w ręku zawszystkie się w rękawie,
Tak w konie się w czerwszy się w pod konieczkie się w rozmieranie się za niemienie
Ogląda w koniec w rozmowę w pod konieczkie się w konie,
Tak się w rozmowanie się w strony w licznych w przecierach się w rozmowy,
Odbiały w obronie w konieczkie w rękę w lecz od zawiedzie,
Obrzecz rozmowy w się w przez zawsze w niemienie,
Obiera w przecie się w las się w strzelcy w przechodzić w korach się w lasu,
Odbierał się w ręku zamku w rozmowy,
Odbiera w konie w podobny w rozmowy, jest w konie w przed nie konieczki

In [22]:
print(evaluate('bę', 200, temperature=1.4))

bęlów orówat;
Chlca teronéj, mougą) łowrzéć skię się! , turek Dządka Soplona,
Po Wuje popan wiorkówlącowém mo zmni;
Rząpie a luże, ry brata: z pirnych
Gdziego do rugiegod
Rzęło, Lonikó, ziejachiłojowa d
